In [1]:
# Import necessary libraries
import os            # Operating system functionality
import glob          # File system path manipulation
import numpy as np   # Numerical operations
#import nd2reader     # Not used in this script
import napari        # Interactive multi-dimensional image viewer
import bigfish       # Library for single-molecule fluorescence imaging
import bigfish.stack as stack        # Functions for processing image stacks
import bigfish.detection as detection  # Spot detection algorithms
import bigfish.multistack as multistack  # Functions for multi-channel stacks
import bigfish.plot as plot          # Plotting utilities
import time          # Time tracking
import pandas as pd  # Data manipulation and analysis
import tifffile as tiff  # Reading and writing TIFF files
from skimage import io   # Image processing functions
import plotly.express as px  # Expressive visualization library
from nis2pyr.convertor import convert_nd2_to_pyramidal_ome_tiff
import math


In [4]:
DIR_ROOT = "./Test/" ##Input Directory with the ND2 files
DIR_TIFFFILES = os.path.join(DIR_ROOT,'TiffFiles') #New subdirectory where processed files will be saved
DIR_MAXPROJ = os.path.join(DIR_TIFFFILES, 'Max_Projections')  # Directory for max projections
DIR_FOVS = os.path.join(DIR_MAXPROJ, 'FOVs')  # Directory for individual FOVs
DIR_COMBINED_FILES = os.path.join(DIR_MAXPROJ, "Combined_Files") # a directory to save combined files if it doesn't exist
DIR_SPOTS = os.path.join(DIR_MAXPROJ, 'Spots') #Save output spot files

# Directories to check and create if they don't exist
dirs_to_create = [DIR_TIFFFILES, DIR_MAXPROJ, DIR_FOVS, DIR_COMBINED_FILES, DIR_SPOTS]

# Loop through each directory
for directory in dirs_to_create:
    # Check if the directory does not exist
    if not os.path.isdir(directory):
        # If it doesn't, create the directory
        os.mkdir(directory)

# Define parameters
voxelval = 110.3752759382
##radiusval = 250.0
radiusval = 2*voxelval

### Saving Files as OME Tiffs

- All images are converte to .tif files and saved in the TiffFiles folder
- Unclear yet how to change the LUTs

In [6]:
# Filter ND2 files and remove those used for testing bleaching
files = sorted([f for f in os.listdir(DIR_ROOT) if f.endswith('.nd2') and 'Bleach' not in f])

# Iterate over each file in the 'files' list (all ND2 files)
for fil in files:
    # Print the directory and filename being processed
    print(os.path.join(DIR_ROOT, fil))
    
    # Convert the ND2 file to pyramidal OME-TIFF format
    # Specify the input ND2 file path, output OME-TIFF file path,
    # and the maximum number of pyramid levels (set to 1 in this case)
    convert_nd2_to_pyramidal_ome_tiff(os.path.join(DIR_ROOT, fil), 
                                    os.path.join(DIR_TIFFFILES, fil.split(".")[0] + '.tif'),
                                    max_levels=1)


./Test/Round00_FOV0000.nd2
Reading ./Test/Round00_FOV0000.nd2
ND2 dimensions: {'Z': 15, 'C': 4, 'Y': 2720, 'X': 2720}; RGB: False; datatype: uint16; legacy: False
Saving pyramidal OME TIFF file ./Test/TiffFiles/Round00_FOV0000.tif
Writing level 0: TZCYXS=(1, 15, 4, 2720, 2720, 1)
Updating OME XML channel names and colors
./Test/Round00_FOV0001.nd2
Reading ./Test/Round00_FOV0001.nd2
ND2 dimensions: {'Z': 15, 'C': 4, 'Y': 2720, 'X': 2720}; RGB: False; datatype: uint16; legacy: False
Saving pyramidal OME TIFF file ./Test/TiffFiles/Round00_FOV0001.tif
Writing level 0: TZCYXS=(1, 15, 4, 2720, 2720, 1)
Updating OME XML channel names and colors
./Test/Round01_FOV0000.nd2
Reading ./Test/Round01_FOV0000.nd2
ND2 dimensions: {'Z': 15, 'C': 4, 'Y': 2720, 'X': 2720}; RGB: False; datatype: uint16; legacy: False
Saving pyramidal OME TIFF file ./Test/TiffFiles/Round01_FOV0000.tif
Writing level 0: TZCYXS=(1, 15, 4, 2720, 2720, 1)
Updating OME XML channel names and colors
./Test/Round01_FOV0001.nd2
Read

### Maximum Intensity Projection and Image Distribution according to FOV 
So all images corresponding to the same FOV but across different cycles are saved in one folder 
- each image for one round is split by its channels and one channel image is saved
- still need to figure out how to manage when there are multiple channels

In [7]:
# List all TIFF files in the input directory and sort them
files = sorted([f for f in os.listdir(DIR_TIFFFILES) if f.endswith('.tif')])

# Define a function to compute maximum intensity projection along the Z-axis
def maximum_intensity_projection(image_stack):
    return np.max(image_stack, axis=0)

# Loop through each TIFF file
for fil in files:
    # Read the OME-TIFF file
    ome_tiff_path = os.path.join(DIR_TIFFFILES, fil)
    image_stack = tiff.imread(ome_tiff_path)
    
    # Compute the maximum intensity projection
    mip = maximum_intensity_projection(image_stack)
    
    # Save the max projection image
    mip_path = os.path.join(DIR_MAXPROJ, "MAX_" + os.path.splitext(fil)[0] + ".tif")
    tiff.imsave(mip_path, mip)
    
    # Extract FOV information from the filename
    FOV = os.path.splitext(fil)[0].split("_")
    
    # Create a directory for the FOV if it does not exist
    dir5 = os.path.join(DIR_FOVS, FOV[1])
    os.makedirs(dir5, exist_ok=True)
    
    # Save each channel of the max projection image as a separate TIFF file
    for chan, channel_image in enumerate(mip, start=1):
        savename = os.path.join(dir5, f"{FOV[0]}_channel_{chan}.tif")
        tiff.imsave(savename, channel_image)



#### Image Registration

### Creating Files combining images from single channel and single FOV across all cycles
- This helps in easy visualization of the data and can also be used for data analysis? 
- the files are saved in the Max_Projection/Combined_Files/ folder

In [14]:
# Get a list of directories in DIR_MAXPROJ
directories = [name for name in os.listdir(DIR_FOVS) if os.path.isdir(os.path.join(DIR_FOVS, name))]
directories.sort()  # Sort the list of directories

# Iterate over each directory
for directory in directories:
    # Create full path to the current directory
    dir_path = os.path.join(DIR_FOVS, directory)
    
    # Get list of TIFF files in the directory and sort them
    tiff_files = sorted([f for f in os.listdir(dir_path) if f.endswith('.tif')])
    
    # Extract channel information from TIFF filenames
    channels = [f.split('_')[2] for f in tiff_files]
    unique_channels = np.unique(channels)
    
    # Iterate over each unique channel
    for channel in unique_channels:
        # Get indices of TIFF files corresponding to the current channel
        indices = [i for i, c in enumerate(channels) if c == channel]
        
        # Initialize array for multichannel image
        multichannel_image = np.zeros((len(indices), channel_image.shape[0], channel_image.shape[1]))
        
        # Iterate over each TIFF file corresponding to the current channel
        for i, index in enumerate(indices):
            # Read TIFF file
            img = tiff.imread(os.path.join(dir_path, tiff_files[index]))
            # Store image in multichannel array
            multichannel_image[i, :, :] = img
        
        # Define filename for combined multichannel image
        savename = os.path.join(DIR_COMBINED_FILES, f"{directory}_channel_{channel}")
        # Save combined multichannel image as TIFF
        tiff.imsave(savename, multichannel_image)


### FISH Spot Detection

##### Detecting threshold within multiple FOVs for all rounds and channels
- This could potentially be used to get a global threshold value
- Alternatively can be used to get individual thresholds for each channel and round

In [26]:
# Record start time
start_time = time.time()

# Threshold values for each channel
thresholds = [18, 18, 18]  # Thresholds for Channels 1, 2, and 3

# Number of cycles for each channel
channel_rounds = [8, 8, 8]

# Number of FOVs to use to detect thresholds
NFOVS = 2

# Get list of TIFF files in the combined files directory and sort them
files = sorted([f for f in os.listdir(DIR_COMBINED_FILES) if f.endswith('.tif')])

# Compute spot radius in pixels
spot_radius_px = detection.get_object_radius_pixel(
                    voxel_size_nm=(voxelval, voxelval), 
                    object_radius_nm=(radiusval, radiusval), 
                    ndim=2)

ths1 = np.zeros((channel_rounds[0]*NFOVS, 1))
ths2 = np.zeros((channel_rounds[1]*NFOVS, 1))
ths3 = np.zeros((channel_rounds[2]*NFOVS, 1))

# Iterate over each channel
for channel, threshold, rounds in zip(range(3), thresholds, channel_rounds):
    e = 0
    for fil in range(NFOVS):
        print(f"--- Start {time.time() - start_time} seconds ---")
        print(fil)
        print(channel)
        print(len(unique_channels))
        
        filename = files[fil * len(unique_channels) + channel]
        img = tiff.imread(os.path.join(DIR_COMBINED_FILES, filename))
        print(filename)
        # Detect spots
        for t in range(img.shape[0]):
            rna = img[t, :, :]
            # LoG filter
            rna_log = stack.log_filter(rna, sigma=spot_radius_px)
            # Local maximum detection
            mask = detection.local_maximum_detection(rna_log, min_distance=spot_radius_px)
            # Thresholding
            threshold_value = detection.automated_threshold_setting(rna_log, mask)
            if channel == 0:
                ths1[e] = threshold_value
            elif channel == 1:
                ths2[e] = threshold_value
            else:
                ths3[e] = threshold_value
            e += 1
    print(f"Finished thresholding for channel {channel + 1} after {time.time() - start_time} seconds ---")


--- Start 0.011390924453735352 seconds ---
0
0
4
FOV0000_channel_1.tif
--- Start 6.827385902404785 seconds ---
1
0
4
FOV0001_channel_1.tif
Finished thresholding for channel 1 after 17.019725799560547 seconds ---
--- Start 17.01987886428833 seconds ---
0
1
4
FOV0000_channel_2.tif
--- Start 28.690464973449707 seconds ---
1
1
4
FOV0001_channel_2.tif
Finished thresholding for channel 2 after 40.99107480049133 seconds ---
--- Start 40.991265058517456 seconds ---
0
2
4
FOV0000_channel_3.tif
--- Start 52.88156795501709 seconds ---
1
2
4
FOV0001_channel_3.tif
Finished thresholding for channel 3 after 64.80895280838013 seconds ---


#### Applying threshold to detect spots and clusters
- spots are saved in the Spots folder as FOV##\_Channel##\_#.csv
- spot clusters are save in the Spots folder as FOV##\_Channel##\_spotclusters_#.csv
- clusters information is save in the Spots folder as FOV##\_Channel##\_clusters\_#.csv


In [49]:
start_time = time.time()

# Compute thresholds
thresholds = [2 * np.median(ths) + 40 for ths in [ths1, ths2, ths3]]

# Get list of TIFF files
files = sorted([each for each in os.listdir(DIR_COMBINED_FILES) if each.endswith('.tif')])
NFOVS = len(files) // len(unique_channels)


for fil in range(NFOVS):
    print("--- Start %s seconds ---" % (time.time() - start_time))
    print(fil)
    for cc, channel in enumerate(unique_channels[0:-1]):
        print("Analysing Channel %s" % cc)
        filename = files[fil * len(unique_channels) + cc]
        print(filename)
        img = tiff.imread(os.path.join(DIR_COMBINED_FILES, filename))
        savedir = os.path.join(DIR_FOVS, "Combined_Files")
        savenamespots = filename.split(".")[0] + ".csv"
        savenamespotscl = filename.split(".")[0] + "_spotclusters.csv"
        savenameclusters = filename.split(".")[0] + "_clusters.csv"

        sp = pd.DataFrame()
        spcl = pd.DataFrame()
        cl = pd.DataFrame()

        # Detect spots
        for t in range(img.shape[0] - 1):
            print("Analysing Round %s" % (t + 1))
            rna = img[t + 1, :, :]
            # LoG filter
            spots = detection.detect_spots(
                images=rna,
                return_threshold=False,
                threshold=thresholds[cc],
                voxel_size=(voxelval, voxelval),  # in nanometer (one value per dimension zyx)
                spot_radius=(radiusval, radiusval)  # in nanometer (one value per dimension zyx)
            )

            spots_post_decomposition, dense_regions, reference_spot = detection.decompose_dense(
                image=np.uint16(rna),
                spots=spots,
                voxel_size=(voxelval, voxelval),
                spot_radius=(radiusval, radiusval),
                alpha=0.75,  # alpha impacts the number of spots per candidate region
                beta=0.9,  # beta impacts the number of candidate regions to decompose
                gamma=15  # gamma the filtering step to denoise the image
            )

            spots_post_clustering, clusters = detection.detect_clusters(
                spots=spots_post_decomposition,
                voxel_size=(int(voxelval), int(voxelval)),
                radius=int(radiusval),
                nb_min_spots=4
            )

            spotspd = pd.DataFrame(spots)
            spclpd = pd.DataFrame(spots_post_clustering)
            clupd = pd.DataFrame(clusters)

            spotspd['round'] = t
            spclpd['round'] = t
            clupd['round'] = t

            sp = pd.concat([sp, spotspd])
            spcl = pd.concat([spcl, spclpd])
            cl = pd.concat([cl, clupd])

        sp.columns = ['Y', 'X', 'round']
        cl.columns = ['Y', 'X', 'nspots', 'index', 'round']
        spcl.columns = ['Y', 'X', 'clusterindex', 'round']

        sp.to_csv(os.path.join(DIR_SPOTS, savenamespots), index=False)
        spcl.to_csv(os.path.join(DIR_SPOTS, savenamespotscl), index=False)
        cl.to_csv(os.path.join(DIR_SPOTS, savenameclusters), index=False)

print("Finished thresholding for channel 1 image %s after %s seconds ---" % (e, time.time() - start_time))

--- Start 0.011821985244750977 seconds ---
0
Analysing Channel 0
FOV0000_channel_1.tif
Analysing Round 1
Analysing Round 2
Analysing Round 3
Analysing Round 4
Analysing Round 5
Analysing Round 6
Analysing Round 7
Analysing Channel 1
FOV0000_channel_2.tif
Analysing Round 1
Analysing Round 2
Analysing Round 3
Analysing Round 4
Analysing Round 5
Analysing Round 6
Analysing Round 7
Analysing Channel 2
FOV0000_channel_3.tif
Analysing Round 1
Analysing Round 2
Analysing Round 3
Analysing Round 4
Analysing Round 5
Analysing Round 6
Analysing Round 7


/Users/asrivath/opt/anaconda3/envs/fish/lib/python3.7/site-packages/bigfish/detection/utils.py:433: UserWarning: Problem occurs during the computation of a reference spot. Not enough (uncropped) spots have been detected.
  UserWarning)


--- Start 95.07934212684631 seconds ---
1
Analysing Channel 0
FOV0001_channel_1.tif
Analysing Round 1
Analysing Round 2
Analysing Round 3
Analysing Round 4
Analysing Round 5
Analysing Round 6
Analysing Round 7
Analysing Channel 1
FOV0001_channel_2.tif
Analysing Round 1
Analysing Round 2
Analysing Round 3
Analysing Round 4
Analysing Round 5
Analysing Round 6
Analysing Round 7
Analysing Channel 2
FOV0001_channel_3.tif
Analysing Round 1
Analysing Round 2
Analysing Round 3
Analysing Round 4
Analysing Round 5
Analysing Round 6
Analysing Round 7
Finished thresholding for channel 1 image 16 after 196.73482179641724 seconds ---


/Users/asrivath/opt/anaconda3/envs/fish/lib/python3.7/site-packages/bigfish/detection/utils.py:433: UserWarning: Problem occurs during the computation of a reference spot. Not enough (uncropped) spots have been detected.
  UserWarning)


### Napari Viewer to visualize overlay of spots on FISH images
- checking signal quality
- see if threshold needs to be adjusted

In [27]:
# Initialize Napari viewer
viewer = napari.Viewer()

# Define parameters
files = sorted([f for f in os.listdir(DIR_COMBINED_FILES) if f.endswith('.tif')])
img = tiff.imread(os.path.join(DIR_COMBINED_FILES, files[0]))
GAP_SIZE = img.shape[1]+10
CHANNELS = pd.unique(pd.DataFrame(np.stack(np.char.split(files, sep="_"), axis=0))[2])
NFOVS = len(files) // len(CHANNELS)

GENES_CY7 = ['Dpp10', 'Dpp8', 'Dst', 'Fat1', 'Itgb4', 'Larp1', 'Larp7']
GENES_CY5 = ['Apob', 'Cdh1', 'Ctnnb1', 'Cyb5r3', 'mTor', 'Net1', 'Pigr']
GENES_CY3 = ['Dync1h1', 'Dync1i2', 'Eif4h', 'Kif1c', 'Kif3b', 'Kif5b', 'None']
color = px.colors.qualitative.Light24

# Initialize variables
SPOTS_CY7 = pd.DataFrame()
SPOTS_CY5 = pd.DataFrame()
SPOTS_CY3 = pd.DataFrame()
#TOTAL_AREA = nearest_square(len(files) *(len(GENES_CY3)+len(GENES_CY5)+len(GENES_CY7))/4)
NUM_ROUNDS = len(GENES_CY7)+1
NUM_CHANNELS = len(CHANNELS)-1
TOTAL_AREA = NFOVS*NUM_CHANNELS
image = np.zeros((GAP_SIZE * NUM_ROUNDS, GAP_SIZE * TOTAL_AREA), dtype=np.int16)
xx = 0
yy = 0

for fov in range(NFOVS):
    for cc, genes in enumerate([GENES_CY7, GENES_CY5, GENES_CY3]):
        image_name = os.path.join(DIR_COMBINED_FILES,files[len(CHANNELS) * fov + cc])
        im2 = tiff.imread(image_name)
        file_name = files[fov * len(CHANNELS) + cc]
        spot = pd.read_csv(os.path.join(DIR_SPOTS, file_name.split(".")[0] + "_spotclusters.csv"))
        for ch_count in range(0,len(genes)+1):
            image[xx * GAP_SIZE:xx * GAP_SIZE + im2.shape[1], yy * GAP_SIZE:yy * GAP_SIZE + im2.shape[1]] = im2[ch_count, :, :]
            spots = spot[spot['round'] == ch_count] 
            spots['Y'] += (xx+1) * GAP_SIZE
            spots['X'] += (yy) * GAP_SIZE
            spots['gene'] = spots['round'].apply(lambda x: genes[x]).values
            if cc == 0:
                SPOTS_CY7 = pd.concat([SPOTS_CY7, spots])
            elif cc == 1:
                SPOTS_CY5 = pd.concat([SPOTS_CY5, spots])
            elif cc == 2:
                SPOTS_CY3 = pd.concat([SPOTS_CY3, spots])
            xx += 1
            if xx > NUM_ROUNDS - 1:
                xx = 0
                yy += 1
print("PAUSE BREAK")
# Add image to viewer
image_layers = [np.uint16(image[::2 ** i, ::2 ** i]) for i in range(int(np.log2(GAP_SIZE)) + 1)]
image_layer.contrast_limits = (0, 65000)

print("PAUSE")
count=0
# Add spots to viewer
all_spots = pd.concat([SPOTS_CY7, SPOTS_CY5, SPOTS_CY3])
#for fov in range(NFOVS):
for genes in [GENES_CY7, GENES_CY5, GENES_CY3]:
    for round in genes:
        cy = np.array(all_spots[all_spots['gene'] == round])[:,:2]
        viewer.add_points(cy,
                        #spots[::2,::2],
                        #spots[::4,::4]],
                        face_color=color[count],
                        size=5, blending='translucent_no_depth', edge_width=0, name=round)
        count = count+1
        if count>len(color)-1:
            count=0
            

/Users/asrivath/opt/anaconda3/envs/fish/lib/python3.7/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/asrivath/opt/anaconda3/envs/fish/lib/python3.7/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/asrivath/opt/anaconda3/envs/fish/lib/python3.7/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

PAUSE BREAK
PAUSE


: 

#### Napari viewer
- View all images in one 

In [5]:
def nearest_square(limit):
    return int(math.ceil(limit ** 0.5))

# Initialize Napari viewer
viewer = napari.Viewer()

# Define parameters
files = sorted([f for f in os.listdir(DIR_COMBINED_FILES) if f.endswith('.tif')])
img = tiff.imread(os.path.join(DIR_COMBINED_FILES, files[0]))
GAP_SIZE = img.shape[1]+50
CHANNELS = pd.unique(pd.DataFrame(np.stack(np.char.split(files, sep="_"), axis=0))[2])
NFOVS = len(files) // len(CHANNELS)

GENES_CY7 = ['Dpp10', 'Dpp8', 'Dst', 'Fat1', 'Itgb4', 'Larp1', 'Larp7']
GENES_CY5 = ['Apob', 'Cdh1', 'Ctnnb1', 'Cyb5r3', 'mTor', 'Net1', 'Pigr']
GENES_CY3 = ['Dync1h1', 'Dync1i2', 'Eif4h', 'Kif1c', 'Kif3b', 'Kif5b', 'None']
color = px.colors.qualitative.Light24

# Initialize variables
SPOTS_CY7 = pd.DataFrame()
SPOTS_CY5 = pd.DataFrame()
SPOTS_CY3 = pd.DataFrame()
TOTAL_AREA = nearest_square(len(files) / 4)
image = np.zeros((GAP_SIZE * TOTAL_AREA, GAP_SIZE * TOTAL_AREA), dtype=np.int16)
xx = 0
yy = 0

for fov in range(NFOVS):
    image_name = files[len(CHANNELS) * fov + 3]
    print(image_name)
    image_loc = os.path.join(DIR_COMBINED_FILES, image_name)
    im2 = tiff.imread(image_loc)
    image[xx * GAP_SIZE:xx * GAP_SIZE + im2.shape[1], yy * GAP_SIZE:yy * GAP_SIZE + im2.shape[1]] = im2[0, :, :]

    for cc, genes in enumerate([GENES_CY7, GENES_CY5, GENES_CY3]):
        file_name = files[fov * len(CHANNELS) + cc]
        spots = pd.read_csv(os.path.join(DIR_SPOTS, file_name.split(".")[0] + "_spotclusters.csv"))
        spots['Y'] += xx * GAP_SIZE
        spots['X'] += yy * GAP_SIZE
        spots['gene'] = spots['round'].apply(lambda x: genes[x]).values
        if cc == 0:
            SPOTS_CY7 = pd.concat([SPOTS_CY7, spots])
        elif cc == 1:
            SPOTS_CY5 = pd.concat([SPOTS_CY5, spots])
        elif cc == 2:
            SPOTS_CY3 = pd.concat([SPOTS_CY3, spots])
    xx += 1
    if xx > TOTAL_AREA - 1:
        xx = 0
        yy += 1

# Add image to viewer
image_layers = [np.uint16(image[::2 ** i, ::2 ** i]) for i in range(int(np.log2(GAP_SIZE)) + 1)]
image_layer.contrast_limits = (0, 65000)

# Concatenate all spots
all_spots = pd.concat([SPOTS_CY7, SPOTS_CY5, SPOTS_CY3])
count=0
# Add spots to viewer
for genes in [GENES_CY7, GENES_CY5, GENES_CY3]:
    for round in genes:
        cy = all_spots[all_spots['gene'] == round]
        spots = np.array(cy)[:, :2]
        viewer.add_points(spots,
                          #spots[::2,::2],
                          #spots[::4,::4]],
                          face_color=color[count],
                          size=5, blending='translucent_no_depth', edge_width=0, name=round)
        count = count+1
        if count>len(color):
            count=0
            
# Export features and all spots
all_spots['gene'].to_csv(os.path.join(DIR_SPOTS, 'features.csv'), index=False)
all_spots.to_csv(os.path.join(DIR_SPOTS, "allspots.csv"), index=False)

feat = tuple(np.array(all_spots['gene']))
point_layer = viewer.add_points(np.array(all_spots)[:, :2], face_color='white', size=5,
                                 blending='translucent_no_depth', edge_width=0, name='All_spots', opacity=0,
                                 features=feat)

point_layer.refresh()


FOV0000_channel_4.tif
0_0
0_1
0_2
FOV0001_channel_4.tif
1_0
1_1
1_2


In [20]:
GENES_CY7 = ['Dpp10', 'Dpp8', 'Dst', 'Fat1', 'Itgb4', 'Larp1', 'Larp7']
GENES_CY5 = ['Apob', 'Cdh1', 'Ctnnb1', 'Cyb5r3', 'mTor', 'Net1', 'Pigr']
GENES_CY3 = ['Dync1h1', 'Dync1i2', 'Eif4h', 'Kif1c', 'Kif3b', 'Kif5b', 'None']
color = px.colors.qualitative.Light24

# Define parameters
files = sorted([f for f in os.listdir(DIR_COMBINED_FILES) if f.endswith('.tif')])
img = tiff.imread(os.path.join(DIR_COMBINED_FILES, files[0]))
GAP_SIZE = img.shape[1]+10
CHANNELS = pd.unique(pd.DataFrame(np.stack(np.char.split(files, sep="_"), axis=0))[2])
NFOVS = len(files) // len(CHANNELS)

# Precompute constants
NUM_ROUNDS = len(GENES_CY7)
TOTAL_AREA = NFOVS * len(CHANNELS) - 1

# Initialize variables
SPOTS_CY7 = []
SPOTS_CY5 = []
SPOTS_CY3 = []
image = np.zeros((GAP_SIZE * NUM_ROUNDS, GAP_SIZE * TOTAL_AREA), dtype=np.int16)

xx=0
yy=0

for fov in range(NFOVS):
    for cc, genes in enumerate([GENES_CY7, GENES_CY5, GENES_CY3]):
        image_name = os.path.join(DIR_COMBINED_FILES, files[len(CHANNELS) * fov + cc])
        im2 = tiff.imread(image_name)
        spot = pd.read_csv(os.path.join(DIR_SPOTS, files[fov * len(CHANNELS) + cc].split(".")[0] + "_spotclusters.csv"))
        
        for ch_count, gene in enumerate(genes):
            image[xx * GAP_SIZE:xx * GAP_SIZE + im2.shape[1], yy * GAP_SIZE:yy * GAP_SIZE + im2.shape[1]] = im2[ch_count, :, :]
            spots = spot[spot['round'] == ch_count] 
            spots['Y'] += (xx + 1) * GAP_SIZE
            spots['X'] += yy * GAP_SIZE
            spots['gene'] = gene
            if cc == 0:
                SPOTS_CY7.append(spots)
            elif cc == 1:
                SPOTS_CY5.append(spots)
            elif cc == 2:
                SPOTS_CY3.append(spots)
            xx += 1
            if xx >= NUM_ROUNDS:
                xx = 0
                yy += 1

# Concatenate dataframes
SPOTS_CY7 = pd.concat(SPOTS_CY7)
SPOTS_CY5 = pd.concat(SPOTS_CY5)
SPOTS_CY3 = pd.concat(SPOTS_CY3)

# Initiate Napari
viewer= napari.Viewer()

# Add image to viewer
image_layers = [np.uint16(image[::2 ** i, ::2 ** i]) for i in range(int(np.log2(GAP_SIZE)) + 1)]
image_layer = viewer.add_image(image_layers)
image_layer.contrast_limits = (0, 65000)

# Add spots to viewer
all_spots = pd.concat([SPOTS_CY7, SPOTS_CY5, SPOTS_CY3])
for gene, color in zip([GENES_CY7, GENES_CY5, GENES_CY3], px.colors.qualitative.Light24):
    for round_gene in gene:
        cy = np.array(all_spots[all_spots['gene'] == round_gene])[:, :2]
        viewer.add_points(cy, face_color=color, size=5, blending='translucent_no_depth', edge_width=0, name=round_gene)


/Users/asrivath/opt/anaconda3/envs/fish/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/asrivath/opt/anaconda3/envs/fish/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/asrivath/opt/anaconda3/envs/fish/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

In [24]:
for i in range(int(np.log2(GAP_SIZE)) + 1):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11


In [7]:
print(np.unique(np.array(all_spots['gene'])))

feat = tuple(np.array(all_spots['gene']))
point_layer = viewer.add_points(np.array(all_spots)[:, :2], face_color='white', size=5,
                                 blending='translucent_no_depth', edge_width=0, name='All_spots', opacity=0,
                                 features=feat)

point_layer.refresh()

['Apob' 'Cdh1' 'Ctnnb1' 'Cyb5r3' 'Dpp10' 'Dpp8' 'Dst' 'Dync1h1' 'Dync1i2'
 'Eif4h' 'Fat1' 'Itgb4' 'Kif1c' 'Kif3b' 'Kif5b' 'Larp1' 'Larp7' 'Net1'
 'None' 'Pigr' 'mTor']
